# Problem Statement

>Imagine you are working as a data scientist at a home electronics company which manufactures state of the art smart televisions. You want to develop a cool feature in the smart-TV that can recognise five different gestures performed by the user which will help users control the TV without using a remote.

>The gestures are continuously monitored by the webcam mounted on the TV. Each gesture corresponds to a specific command:

>Thumbs up:  Increase the volume

>Thumbs down: Decrease the volume

>Left swipe: 'Jump' backwards 10 seconds

>Right swipe: 'Jump' forward 10 seconds  

>Stop: Pause the movie


# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [3]:
#cv2 installation
#pip install cv2
#!pip install opencv-python#
'''
!pip install opencv-python==4.2.0.34
'''

'\n!pip install opencv-python==4.2.0.34\n'

In [4]:
#importing packages
import numpy as np
import os
from scipy.misc import imread, imresize
#import cv2
import datetime
import os

import warnings
warnings.filterwarnings('ignore')

We set the random seed so that the results don't vary drastically.

In [5]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
#tf.set_random_seed(30)
tf.random.set_seed(30)

In [6]:
## Mounting google drive
'''
from google.colab import drive
drive.mount('/content/drive')
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [7]:
# Loading videos folder names
#train_doc = np.random.permutation(open('/content/drive/MyDrive/Project_data/train.csv').readlines())
#val_doc = np.random.permutation(open('/content/drive/MyDrive/Project_data/val.csv').readlines())

train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
#train_doc = np.random.permutation(open('C:/Studies/Upgrad - PGC in ML and DL/Course5 Deep Learning/Group Case Study1 - Neural Networks Project - Gesture Recognition/Project_data/train.csv').readlines())
#val_doc = np.random.permutation(open('C:/Studies/Upgrad - PGC in ML and DL/Course5 Deep Learning/Group Case Study1 - Neural Networks Project - Gesture Recognition/Project_data/val.csv').readlines())
batch_size = 10#experiment with the batch size
#batch size of upto 100 is acceptable but getting lower validation accuracy

In [8]:
#defining image height and width using which we can standardize size of all images
img_height=120
img_width=160

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [9]:
#function to normalize each individual image
def normalize_img(image):
    normalized_image= (image - np.min(image))/(np.max(image)- np.min(image))
    return normalized_image

In [10]:
#generator function that creates batched of input data and corresponding class matrices
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0, 30,2)]#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size# calculating the number of full batches
        
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),img_height,img_width,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    #image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    if image.shape[0]==120 & image.shape[1]==160:
                        resized_img=image
                    else:
                        resized_img=imresize(image,(img_height,img_width,3))
                        #resized_img=cv2.resize(image, (img_width, img_height))#In cv2.imresize, we take width followed by height
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = normalize_img(resized_img[:, : , 0])#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = normalize_img(resized_img[:, : , 1])#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = normalize_img(resized_img[:, : , 2])#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        
        
        # write the code for the remaining data points which are left after full batches
        rem_batch_size=len(folder_list)%batch_size
        batch_data = np.zeros((rem_batch_size,len(img_idx),img_height,img_width,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((rem_batch_size,5)) # batch_labels is the one hot representation of the output
        for folder in range(rem_batch_size): # iterate over the batch_size
            imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                #image = cv2.imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                if image.shape[0]==120 & image.shape[1]==160:
                    resized_img=image
                else:
                    resized_img=imresize(image,(img_height,img_width,3))
                    #resized_img=cv2.resize(image, (img_width, img_height))

                #crop the images and resize them. Note that the images are of 2 different shape 
                #and the conv3D will throw error if the inputs in a batch have different shapes

                batch_data[folder,idx,:,:,0] = normalize_img(resized_img[:, : , 0])#normalise and feed in the image
                batch_data[folder,idx,:,:,1] = normalize_img(resized_img[:, : , 1])#normalise and feed in the image
                batch_data[folder,idx,:,:,2] = normalize_img(resized_img[:, : , 2])#normalise and feed in the image

            batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
        yield batch_data, batch_labels 

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [11]:
curr_dt_time = datetime.datetime.now()
#train_path = '/content/drive/MyDrive/Project_data/train'
#val_path = '/content/drive/MyDrive/Project_data/val'

train_path = 'Project_data/train'
val_path = 'Project_data/val'
#train_path = 'C:/Studies/Upgrad - PGC in ML and DL/Course5 Deep Learning/Group Case Study1 - Neural Networks Project - Gesture Recognition/Project_data/train'
#val_path = 'C:/Studies/Upgrad - PGC in ML and DL/Course5 Deep Learning/Group Case Study1 - Neural Networks Project - Gesture Recognition/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

We will build 3d convolution model first, followed by CNN+RNN

**Convolution 3D model**

In [12]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, BatchNormalization, Activation, Dropout, Conv2D
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

# building 3d convolution model
input_shape=(15,img_height,img_width,3)

conv_3D_model = Sequential()

conv_3D_model.add(Conv3D(16, 
                 kernel_size=(5,5,5),
                 input_shape=input_shape,
                 padding='same', name="conv1"))
conv_3D_model.add(Activation('relu'))
conv_3D_model.add(BatchNormalization())
conv_3D_model.add(MaxPooling3D(pool_size=(2,2,2)))

conv_3D_model.add(Conv3D(32, 
                 kernel_size=(3,3,3),
                 padding='same',name="conv2"))
conv_3D_model.add(Activation('relu'))
conv_3D_model.add(BatchNormalization())
conv_3D_model.add(MaxPooling3D(pool_size=(2,2,2)))

conv_3D_model.add(Conv3D(64, 
                 kernel_size=(1,3,3), 
                 padding='same',name="conv3"))
conv_3D_model.add(Activation('relu'))
conv_3D_model.add(BatchNormalization())
conv_3D_model.add(MaxPooling3D(pool_size=(2,2,2)))
#conv_3D_model.add(BatchNormalization())
conv_3D_model.add(Dropout(0.25))

#conv_3D_model.add(MaxPooling3D(pool_size=(2,2,2)))
conv_3D_model.add(Flatten())
conv_3D_model.add(Dense(128, activation='relu'))
conv_3D_model.add(BatchNormalization())
conv_3D_model.add(Dropout(0.40))

conv_3D_model.add(Dense(64, activation='relu'))
conv_3D_model.add(BatchNormalization())
conv_3D_model.add(Dropout(0.40))
#softmax layer
conv_3D_model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [13]:
optimiser = optimizers.Adam(0.001)#write your optimizer
conv_3D_model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (conv_3D_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 15, 120, 160, 16)  6016      
_________________________________________________________________
activation (Activation)      (None, 15, 120, 160, 16)  0         
_________________________________________________________________
batch_normalization (BatchNo (None, 15, 120, 160, 16)  64        
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 7, 60, 80, 16)     0         
_________________________________________________________________
conv2 (Conv3D)               (None, 7, 60, 80, 32)     13856     
_________________________________________________________________
activation_1 (Activation)    (None, 7, 60, 80, 32)     0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 60, 80, 32)     1

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [14]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

'''
bat_dat,bat_lab=next(train_generator)
print(bat_dat.shape)#gives (10, 15, 120, 160, 3) size with 10 sequences per batch and 15 images per sequence. 120 is height and 160 is width. 3 represents channels.
print(bat_lab)#10*5 matrix. 10 represents number of sequences. 5 represents classes per sequence.
'''

'\nbat_dat,bat_lab=next(train_generator)\nprint(bat_dat.shape)#gives (10, 15, 120, 160, 3) size with 10 sequences per batch and 15 images per sequence. 120 is height and 160 is width. 3 represents channels.\nprint(bat_lab)#10*5 matrix. 10 represents number of sequences. 5 represents classes per sequence.\n'

In [15]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

W0425 17:05:16.595009 139960078784320 callbacks.py:1071] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
W0425 17:05:16.596150 139960078784320 callbacks.py:2207] `epsilon` argument is deprecated and will be removed, use `min_delta` instead.


The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [16]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1


if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [15]:
conv_3D_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

W0425 10:34:43.663279 139708574598976 deprecation.py:323] From <ipython-input-15-9f69886f7c95>:3: Model.fit_generator (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
Please use Model.fit, which supports generators.


Source path =  Project_data/train ; batch size = 10
Epoch 1/30
67/67 [==============================] - ETA: 0s - loss: 2.0066 - categorical_accuracy: 0.3394Source path =  Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-04-2510_34_40.410763/model-00001-2.00657-0.33937-4.15404-0.17000.h5
67/67 [==============================] - 53s 785ms/step - loss: 2.0066 - categorical_accuracy: 0.3394 - val_loss: 4.1540 - val_categorical_accuracy: 0.1700 - lr: 0.0010
Epoch 2/30
67/67 [==============================] - ETA: 0s - loss: 1.6107 - categorical_accuracy: 0.4223
Epoch 00002: saving model to model_init_2021-04-2510_34_40.410763/model-00002-1.61075-0.42232-5.91296-0.19000.h5
67/67 [==============================] - 52s 770ms/step - loss: 1.6107 - categorical_accuracy: 0.4223 - val_loss: 5.9130 - val_categorical_accuracy: 0.1900 - lr: 0.0010
Epoch 3/30
67/67 [==============================] - ETA: 0s - loss: 1.2690 - categorical_accuracy: 0.5113
Epoch 00003: sav

Epoch 22/30
67/67 [==============================] - ETA: 0s - loss: 0.6740 - categorical_accuracy: 0.7557
Epoch 00022: saving model to model_init_2021-04-2510_34_40.410763/model-00022-0.67405-0.75566-0.88617-0.69000.h5
67/67 [==============================] - 50s 751ms/step - loss: 0.6740 - categorical_accuracy: 0.7557 - val_loss: 0.8862 - val_categorical_accuracy: 0.6900 - lr: 1.0000e-05
Epoch 23/30
67/67 [==============================] - ETA: 0s - loss: 0.7093 - categorical_accuracy: 0.7360
Epoch 00023: saving model to model_init_2021-04-2510_34_40.410763/model-00023-0.70933-0.73605-0.89394-0.68000.h5
67/67 [==============================] - 51s 765ms/step - loss: 0.7093 - categorical_accuracy: 0.7360 - val_loss: 0.8939 - val_categorical_accuracy: 0.6800 - lr: 1.0000e-05
Epoch 24/30
67/67 [==============================] - ETA: 0s - loss: 0.6569 - categorical_accuracy: 0.7466
Epoch 00024: saving model to model_init_2021-04-2510_34_40.410763/model-00024-0.65686-0.74661-0.89739-0.680

**CNN+RNN model**

We will create a CNN model(with resnet transfer learning) and feed it's extracted features to RNN

In [17]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Dense, GRU, Flatten, TimeDistributed, BatchNormalization, Activation, Dropout, Conv2D
from keras.layers.convolutional import Conv3D, MaxPooling3D, MaxPooling2D
#from keras.models import Sequential

In [24]:
#resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))
resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(img_height,img_width,3))
for layer in resnet.layers[:-5]:
    layer.trainable=False
model = Sequential()
model.add(TimeDistributed(resnet,input_shape=(15,img_height,img_width,3)))

model.add(TimeDistributed(Conv2D(16, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
#model.add(Dropout(0.25))


model.add(TimeDistributed(Conv2D(32, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

#model.add(Dropout(0.25))

model.add(TimeDistributed(Flatten()))


model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
'''
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
'''
model.add(GRU(128, return_sequences=False))
model.add(Dense(5, activation='softmax')) # using Softmax as last layer

#model = Sequential([resnet,Flatten(),Dense(64,activation='relu'),Dense(5,activation='softmax')])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_8 (TimeDist (None, 15, 4, 5, 2048)    23587712  
_________________________________________________________________
time_distributed_9 (TimeDist (None, 15, 4, 5, 16)      294928    
_________________________________________________________________
time_distributed_10 (TimeDis (None, 15, 4, 5, 16)      64        
_________________________________________________________________
time_distributed_11 (TimeDis (None, 15, 2, 2, 16)      0         
_________________________________________________________________
time_distributed_12 (TimeDis (None, 15, 2, 2, 32)      4640      
_________________________________________________________________
time_distributed_13 (TimeDis (None, 15, 2, 2, 32)      128       
_________________________________________________________________
time_distributed_14 (TimeDis (None, 15, 1, 1, 32)     

In [25]:
'''
#resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))
resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(img_height,img_width,3))

for layer in resnet.layers[:-5]:
    layer.trainable=False


cnn =Sequential([resnet])
cnn.add(Conv2D(64,(2,2),strides=(1,1),padding='same', activation='relu'))
cnn.add(Conv2D(16,(3,3),strides=(1,1),padding='same', activation='relu'))
cnn.add(Flatten())

#resnet.summary()
cnn.summary()
'''

"\n#resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))\nresnet = ResNet50(include_top=False,weights='imagenet',input_shape=(img_height,img_width,3))\n\nfor layer in resnet.layers[:-5]:\n    layer.trainable=False\n\n\ncnn =Sequential([resnet])\ncnn.add(Conv2D(64,(2,2),strides=(1,1),padding='same', activation='relu'))\ncnn.add(Conv2D(16,(3,3),strides=(1,1),padding='same', activation='relu'))\ncnn.add(Flatten())\n\n#resnet.summary()\ncnn.summary()\n"

In [26]:
'''
model = Sequential()
#model.add(TimeDistributed(cnn,input_shape=(30,224,224,3)))
model.add(TimeDistributed(cnn,input_shape=(15,img_height,img_width,3)))
#model.add(GRU(16,input_shape=(None,15,127,127,8),return_sequences=True))
model.add(GRU(16,input_shape=(None,15,32),return_sequences=True))
model.add(GRU(8))
model.add(Dense(5,activation='softmax'))
model.summary()
'''

"\nmodel = Sequential()\n#model.add(TimeDistributed(cnn,input_shape=(30,224,224,3)))\nmodel.add(TimeDistributed(cnn,input_shape=(15,img_height,img_width,3)))\n#model.add(GRU(16,input_shape=(None,15,127,127,8),return_sequences=True))\nmodel.add(GRU(16,input_shape=(None,15,32),return_sequences=True))\nmodel.add(GRU(8))\nmodel.add(Dense(5,activation='softmax'))\nmodel.summary()\n"

In [27]:
#opt = tf.keras.optimizers.SGD(lr=0.001,...) 
#model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
optimiser = optimizers.Adam(0.001)
model.compile(loss='categorical_crossentropy',optimizer=optimiser,metrics=['categorical_accuracy'])

In [28]:
num_epochs=20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 1.5665 - categorical_accuracy: 0.2896
Epoch 00001: saving model to model_init_2021-04-2517_05_02.433584/model-00001-1.56652-0.28959-1.97213-0.26000.h5
67/67 [==============================] - 55s 817ms/step - loss: 1.5665 - categorical_accuracy: 0.2896 - val_loss: 1.9721 - val_categorical_accuracy: 0.2600 - lr: 0.0010
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 1.4047 - categorical_accuracy: 0.3575
Epoch 00002: saving model to model_init_2021-04-2517_05_02.433584/model-00002-1.40469-0.35747-1.64569-0.31000.h5
67/67 [==============================] - 52s 770ms/step - loss: 1.4047 - categorical_accuracy: 0.3575 - val_loss: 1.6457 - val_categorical_accuracy: 0.3100 - lr: 0.0010
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 1.3556 - categorical_accuracy: 0.4268
Epoch 00003: saving model to model_init_2021-04-2517_05_02.433584/model-00003-1.35562-0.42685-1.74842-0.32000.h5
67/67